In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from lightgbm.sklearn import LGBMClassifier

In [2]:
trade_area = pd.read_csv("../../data/Smoothie King/smoothie_king_trade_area_variables.csv")
poi = pd.read_csv('../../data/Smoothie King/smoothie_king_poi_variables.csv')
stores = pd.read_csv('../../data/Smoothie King/smoothie_king_stores.csv')
merged = stores.merge(trade_area, left_on="store", right_on="store_num").merge(poi)
merged = merged.drop(columns=["store_num", "country_code"])
merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,medhhinc_dma,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,85075.0,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,58200.0,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,66049.0,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,62779.0,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,76139.0,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,76139.0,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,82668.0,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,72903.0,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,66003.0,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [3]:
to_remove = [col for col in merged.columns.tolist() if "_p_" in col]
merged = merged.drop(columns=to_remove)
merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,medhhinc_dma,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,85075.0,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,58200.0,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,66049.0,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,62779.0,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,76139.0,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,76139.0,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,82668.0,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,72903.0,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,66003.0,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [4]:
print(len(merged["dma_name"].unique()))
len(merged["state_name"].unique())
len(merged["cbsa_name"].unique())

109


171

In [5]:
miss = merged.columns[merged.isnull().any()].tolist()
miss

['inrix_breakfast_ns',
 'inrix_lunch_ns',
 'inrix_afternoon_ns',
 'inrix_dinner_ns',
 'inrix_night_ns',
 'inrix_overnight_ns',
 'inrix_breakfast_ew',
 'inrix_lunch_ew',
 'inrix_afternoon_ew',
 'inrix_dinner_ew',
 'inrix_night_ew',
 'inrix_overnight_ew']

In [6]:
merged = merged.drop(columns=["store", "longitude", "latitude", "state_name", "cbsa_name", "dma_name"])
# merged = merged.drop(columns=miss)
merged = merged.dropna()

In [7]:
merged

,category,market_size,store_density,medhhinc_dma,medhhinc_1mi,age0018_ta,age65pl_ta,age85pl_ta,asian_ta,avg_faminc_ta,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SHOPPING,Large Metro (2),Light Suburban,85075.0,83846.0,21456.0,6709.0,372.0,4092.9205,108837.0,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SHOPPING,Medium City (4),Light Suburban,58200.0,50388.0,12717.0,8094.0,704.0,1723.0000,80752.0,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,HOME,Very Large Metro (1),Light Suburban,66049.0,59999.0,10127.0,8873.0,677.0,1082.0001,106893.0,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,TRAVEL,Very Large Metro (1),Suburban,62779.0,97439.0,8623.0,8821.0,775.0,850.0585,134373.0,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,WORK,Very Large Metro (1),Light Suburban,76139.0,56156.0,17652.0,7902.0,348.0,4425.0000,102367.0,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,HOME,Very Large Metro (1),Exurban,76139.0,32999.0,22186.0,12357.0,900.0,689.0000,66363.0,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,OTHER,Medium City (4),Light Suburban,82668.0,55848.0,13060.0,6761.0,629.0,1907.5900,95226.0,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SHOPPING,Very Large Metro (1),Exurban,72903.0,51204.0,14791.0,7410.0,602.0,929.9680,89271.0,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SHOPPING,Medium City (4),Exurban,66003.0,50154.0,27431.0,14186.0,1158.0,3537.1850,67117.0,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [8]:
train_df, test_df = train_test_split(merged, test_size=0.1, random_state=42)
X_train = train_df.drop(columns=["category"])
y_train = train_df["category"]
X_test = test_df.drop(columns=["category"])
y_test = test_df["category"]

In [9]:
# categorical_features = ["store_density"]
ordinal_features = ["market_size", "store_density"]
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

In [10]:
market_levels = [
    "Small Town (6)",
    "Small City (5)",
    "Medium City (4)",
    "Large City (3)",
    "Large Metro (2)",
    "Very Large Metro (1)"
]
density_levels = [
    "Rural",
    "Exurban",
    "Suburban",
    "Light Suburban",
    "Light Urban",
    "Urban",
    "Super Urban"
]

In [11]:
ordinal_transformer = OrdinalEncoder(categories=[market_levels, density_levels], dtype=int)

preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy="constant", fill_value=0), StandardScaler()), numeric_features),
    (ordinal_transformer, ordinal_features),
)

In [12]:
X_transformed = preprocessor.fit_transform(X_train)
pd.DataFrame(X_transformed)

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.505518,1.603831,0.240326,-0.922216,-0.701716,-0.432801,-0.408111,-0.359667,1.994195,-0.707100,...,0.323607,-0.657455,-0.775485,-0.300743,-0.110049,0.583815,-0.365336,-0.562677,5.0,1.0
1,0.505518,-0.993985,0.204838,-0.509750,-0.735201,0.197289,-1.214297,-1.005594,2.327336,-0.183032,...,-0.467842,1.081276,0.334982,-0.078099,-0.431555,-0.400870,-0.379246,-0.562677,5.0,1.0
2,-1.467147,-2.192245,-1.298315,-1.320784,-0.786309,-0.696760,-1.154591,-0.602471,0.056524,-1.478467,...,1.901325,-0.657455,-0.775485,-0.857354,-0.994191,-1.019566,1.059455,-0.562677,0.0,1.0
3,0.144807,0.340139,0.711398,-0.341735,-0.380969,0.011464,1.227123,0.922776,-0.773621,-0.068242,...,0.096915,-0.657455,-0.674534,-0.523388,-0.270802,-0.470582,-0.385665,-0.562677,3.0,1.0
4,-0.274878,-0.246878,-0.073236,1.065911,1.862501,-0.434474,0.038023,-0.057034,-0.813717,0.600470,...,-0.134381,-0.657455,0.234031,0.033223,0.238250,0.182970,-0.447077,1.379677,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.689684,2.302042,-0.616581,-0.898143,-0.846229,1.222654,1.309654,1.396476,-0.269640,-0.729441,...,-0.533424,0.501699,0.032127,-0.078099,-0.217218,-0.505438,-0.009270,-0.562677,5.0,1.0
703,2.097560,1.529740,-0.202550,0.745517,0.857962,0.770259,1.637960,1.713950,-0.442362,0.620885,...,-0.487917,1.950641,1.041644,1.313429,1.711821,1.986774,-0.446378,0.454747,5.0,2.0
704,-0.183700,-1.028735,-0.513033,-0.407005,-0.113092,-0.552016,-1.105428,-1.161848,-0.737739,-0.701707,...,-0.534673,0.211910,-0.169776,-0.523388,-0.726269,-0.923712,1.649894,-0.562677,1.0,1.0
705,-0.095571,0.530611,-0.670867,-1.206624,-1.010127,0.213299,-0.003261,-0.288191,0.472808,-1.073042,...,-0.023855,-0.657455,0.234031,0.311529,0.559756,0.453105,-0.445582,0.454747,5.0,1.0


In [13]:
transformed_col_names = (
    numeric_features + ordinal_features 
)

## LGBM

In [14]:
pipe_lgbm = make_pipeline(
    preprocessor,
    LGBMClassifier(random_state=42, learning_rate=0.1, reg_alpha=0.1, reg_lambda=0.5, objective="multiclass")
)

In [ ]:
pipe_lgbm.fit(X_train, y_train)

In [ ]:
pipe_lgbm.score(X_test, y_test)

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].feature_importances_

In [ ]:
data = {
    "Importance": pipe_lgbm.named_steps["lgbmclassifier"].feature_importances_,
}
pd.DataFrame(data=data, index=transformed_col_names,).sort_values(
    by="Importance", ascending=False
)[:10]

In [ ]:
results = {}
results["Logistic Regression"] = [pipe_lr.score(X_train, y_train), pipe_lr.score(X_test, y_test)]
results["Light GBM"] = [pipe_lgbm.score(X_train, y_train), pipe_lgbm.score(X_test, y_test)]
pd.DataFrame(results, index=["Train score", "Test score"]).T

In [ ]:
corr_lookup_dict = {}

In [ ]:
correlation = merged.corr(numeric_only=True)
np.fill_diagonal(correlation.values, 0)
correlation

In [ ]:
high_corr = correlation.loc[:, (correlation > 0.98).any(axis=0)]
high_corr[high_corr["age0018_ta"] > 0.98]

In [ ]:
for col_name in high_corr.columns.tolist():
    corr_lookup_dict[col_name] = {}
    for index, row in high_corr[high_corr[col_name] > 0.98].iterrows():
        corr_lookup_dict[col_name][index] = row[col_name]

In [ ]:
for k, v in corr_lookup_dict.items():
    print(k)
    print("Is correlated to")
    print(v)
    print("-------------------")